In [23]:
#importando bibliotecas necessárias no projeto
from sklearn import svm
from keras.models import Sequential
from keras.utils import plot_model
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.python.client import device_lib
from sklearn.svm import SVC
from keras import utils as np_utils
from keras import backend
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from re import search
import numpy as np
from subprocess import getoutput as gop
import glob
import pandas as pd
import csv

### Carregando conjuntos de treino e teste

In [24]:
# identificando pastas
folders = {
    'small': 'dataset/small',
    'large_train': 'dataset/large_train',
    'large_test': 'dataset/large_test',
}

def load_data(dataset):
    ch_names = []
    create_ch_name = False
    
    data_dir = gop('ls {}'.format(folders[dataset])).split('\n')
    # 1ª dimensão dos dados contendo os sujeitos
    subjects = list()
    subjects_alc = list()
    subjects_ctrl = list()
    
    for types in data_dir:
        files = gop('ls {}/{}'.format(folders[dataset], types)).split('\n')
        # 2ª dimensão dos dados contendo as sessões (trials)
        trials = list()
        is_alc = True
        
        for f in files:
            arquivo = open('{}/{}/{}'.format(folders[dataset], types, f))
            text = arquivo.readlines()
            
            alc = search('co2a', text[0])
            ctrl = search('co2c', text[0])
            
            if ctrl:
                is_alc = False
            # 3ª dimensão dos dados contendo os canais (eletrodos)
            chs = list()
            # 4ª dimensão dos dados contendo os valores em milivolts
            values = list()
            for line in text:
                t = search('(?P<ch_name>\w{1,3}) chan \d{1,2}', line)
                p = search('^\d{1,3}\ \w{1,3}\ \d{1,3}\ (?P<value>.+$)', line)
                                    
                if p:
                    values.append(float(p.group('value')))
                # mudou para outro eletrodo
                elif t:
                    if values:
                        chs.append(values)
                        values = list()
                    if not create_ch_name:
                        ch_names.append(t.group('ch_name').lower())
            
            create_ch_name = True
            chs.append(values)
            trials.append(chs)
            arquivo.close()
            
        if is_alc:
            subjects_alc.append(trials)
            md_alc = np.average(trials, axis=0)
        else:
            subjects_ctrl.append(trials)
            md_ctrl = np.average(trials, axis=0)
            
    data_alc = np.array(subjects_alc)
    data_ctrl = np.array(subjects_ctrl)
    
#     data.tofile('./dataset_csv/small_data.csv', sep=',', newline='')   
    return data_alc, md_alc, data_ctrl, md_ctrl, ch_names


In [25]:
data_alc, md_alc, data_ctrl, md_ctrl, ch_names = load_data('large_train')

print('**** Large Train ****\n')
print('Alcoólatras: ', data_alc)
print('\n**** Fim Large Train ****\n')

**** Large Train ****

Alcoólatras:  [[[[-8.9210e+00 -8.4330e+00 -2.5740e+00 ...  4.2620e+00  5.7270e+00
     8.1690e+00]
   [ 8.3400e-01  3.2760e+00  5.7170e+00 ...  2.7870e+00  9.6230e+00
     1.3041e+01]
   [-1.9847e+01 -1.2522e+01  1.1490e+00 ...  7.0090e+00  1.1403e+01
     1.2868e+01]
   ...
   [-4.7800e-01 -9.6600e-01 -9.6600e-01 ...  4.4050e+00  3.9160e+00
     2.9400e+00]
   [-8.9010e+00 -7.9240e+00 -3.0420e+00 ...  4.7710e+00  6.2360e+00
     8.6770e+00]
   [-5.6360e+00 -2.7060e+00  1.6890e+00 ...  1.5849e+01  1.6337e+01
     1.4872e+01]]

  [[ 9.0640e+00  6.6220e+00  4.6690e+00 ...  3.5919e+01  3.3966e+01
     3.3966e+01]
   [ 8.3400e-01 -1.1190e+00 -1.6070e+00 ...  2.9154e+01  2.6225e+01
     2.3295e+01]
   [ 9.9280e+00  1.0417e+01  7.9750e+00 ...  5.5340e+00  2.1160e+00
     5.5340e+00]
   ...
   [ 1.6300e-01  6.5100e-01  1.1390e+00 ... -3.2600e-01  1.6300e-01
     6.5100e-01]
   [ 9.2370e+00  7.7720e+00  5.8190e+00 ...  3.5604e+01  3.3651e+01
     3.3651e+01]
   [ 8.2700e

### Modelo de rede neural

In [26]:
'''model = Sequential()
num_input = lg_train_alc.shape[1]

model.add(Dense(units=100, activation='relu', input_dim=num_input))
model.add(Dense(units=59, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = model.fit(md_alc_train, md_ctrl_train, validation_split=0.30, epochs=50, batch_size=50, verbose=1)

loss_and_metrics = model.evaluate(md_alc_test, md_ctrl_test, batch_size=50)
print("\n Taxa de acerto: %.2f%%" % (loss_and_metrics[1]*100))

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

print("\n Função de Custo: %.2f%%" % (loss_and_metrics[0]))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()'''

'model = Sequential()\nnum_input = lg_train_alc.shape[1]\n\nmodel.add(Dense(units=100, activation=\'relu\', input_dim=num_input))\nmodel.add(Dense(units=59, activation=\'relu\'))\nmodel.add(Dense(1, activation=\'sigmoid\'))\n\nmodel.compile(loss=\'binary_crossentropy\',\n              optimizer=\'adam\',\n              metrics=[\'accuracy\'])\n\n\nhistory = model.fit(md_alc_train, md_ctrl_train, validation_split=0.30, epochs=50, batch_size=50, verbose=1)\n\nloss_and_metrics = model.evaluate(md_alc_test, md_ctrl_test, batch_size=50)\nprint("\n Taxa de acerto: %.2f%%" % (loss_and_metrics[1]*100))\n\nplt.plot(history.history[\'acc\'])\nplt.plot(history.history[\'val_acc\'])\nplt.title(\'Model accuracy\')\nplt.ylabel(\'Accuracy\')\nplt.xlabel(\'Epoch\')\nplt.legend([\'Train\', \'Test\'], loc=\'upper left\')\nplt.show()\n\nprint("\n Função de Custo: %.2f%%" % (loss_and_metrics[0]))\nplt.plot(history.history[\'loss\'])\nplt.plot(history.history[\'val_loss\'])\nplt.title(\'Model loss\')\nplt.